In [14]:
import pandas as pd
import numpy as np

# define the name of the csv file you want to read in
accounts = './data/accounts.csv'
playbacks = './data/playbacks.csv'
subscriptions = './data/subscriptions.csv'

# PLAYBACKS

In [15]:
# Read playbacks
df_playbacks = pd.read_csv(playbacks)
df_playbacks.columns = df_playbacks.columns.str.lower()
display(df_playbacks.shape)
display(df_playbacks.date_start.tail())
# one account can have multiple subscriptions
# what about pausing the movie
# IP + User_Agent-> check for multiple persons per subscriptions
    # IP/Account
# average playback duration
# cancelled session?

(139569, 7)

139564    2020-10-21 13:21:30
139565    2020-10-21 12:35:04
139566    2020-10-21 12:05:14
139567    2020-10-21 11:46:11
139568    2020-10-21 11:18:58
Name: date_start, dtype: object

In [16]:
df_playbacks.isna().sum()

subscription_key    0
account_key         0
movie_id            0
date_start          0
playback_time       0
user_agent          0
ip_hash             0
dtype: int64

In [17]:
df_playbacks.describe()
# have a closer look at max playback times

,movie_id,playback_time
count,139569.000000,139569.00000
mean,712.801589,6442.57679
std,346.698325,8191.70516
min,8.000000,960.00000
25%,453.000000,4680.00000
50%,745.000000,5760.00000
75%,977.000000,6660.00000
max,1344.000000,472920.00000


In [19]:
#DIFFERENTIATION BETWEEN DESKTOP/MOBILE
#TODO Create 'DESKTOP vs. MOBILE' column
#TODO filmingo app users dedicated column?!

#df_playbacks.user_agent.count()
#5387 unique values
#139.569 count

#SEARCH FOR 'Macintosh'
#df_playbacks[df_playbacks.user_agent.str.contains('Macintosh', na=False)].user_agent.count()
#64.401 count ~50%

#SEARCH FOR 'Windows'
#df_playbacks[df_playbacks.user_agent.str.contains('Windows', na=False)].user_agent.count()
#41.475

#DESKTOP Views
df_playbacks[df_playbacks.user_agent.str.contains(('Windows|Macintosh|TV|Linux'), na=False)].user_agent.count()
#117.081 Desktop playbacks

# Mobile Views
#df_playbacks[df_playbacks.user_agent.str.contains(('Android|iOS|iPhone|iPad'), na=False)].user_agent.count()
# 22.800 Mobile views

df_playbacks[df_playbacks.user_agent.str.contains(('Linux'), na=False)].user_agent.unique()
df_playbacks[df_playbacks.user_agent.str.contains(('Darwin'), na=False)].user_agent.count()

#HOW MANY PEOPLE ARE USING THE APP
df_playbacks[~df_playbacks.user_agent.str.contains('filmingo', na=False)].user_agent.count()
#13.173 playbacks via the app



126396

In [27]:
#Let's create the new columns
df_playbacks.loc[df_playbacks['user_agent'].str.contains('Windows|Macintosh|TV|Linux|Darwin|CrOS'), 'device'] = 'desktop'
df_playbacks.loc[df_playbacks['user_agent'].str.contains('Android|iOS|iPhone|iPad'), 'device'] = 'mobile'
df_playbacks.loc[df_playbacks['user_agent'].str.contains('filmingo'), 'app user'] = 'yes'
df_playbacks.loc[~df_playbacks['user_agent'].str.contains('filmingo'), 'app user'] = 'no'
df_playbacks.groupby('device').count()
#df_playbacks[~df_playbacks.device.str.contains(('Android|iOS|iPhone|iPad|Windows|Macintosh|TV|Linux'), na=False)].user_agent.count()



,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,app user
device,,,,,,,,
desktop,116734,116734,116734,116734,116734,116734,116734,116734
mobile,22800,22800,22800,22800,22800,22800,22800,22800


In [28]:
df_playbacks.head()

,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,device,app user
0,9679626b6cc1765f8bc8da98fda2d8e8f28f40736fd777...,024ebde74e23ca924cff0a3989e7f87efe2b2cbcd7ad2c...,1189,2022-09-30 23:55:31,12420,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,0b14aa31d5635ee899be928089fd810c655a7ec7785911...,desktop,no
1,513d006472c99ef047c64bafad57a39906ad5c618505b1...,8968e3851322a89c265a0ec4c81341361e3b0953502d65...,1327,2022-09-30 23:48:36,1440,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4...,c65100e7d48b9e06ca6d89ffe594066553ea63ddc70ee1...,desktop,no
2,cac0c77d3aedc55303b8dfea314052eba5dcf4817e4962...,8a2d7ac64595778ed5fe7edb2dd43f843f9879d773df1a...,212,2022-09-30 23:47:33,1080,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,db17d38bf724c377dcd463f7ad8df8279b6122c8890773...,desktop,no
3,a35503448084642ac8651a0f482db410ccdd7c7716df1d...,7f654ed491a8d1c5b6eb54ea98f6b30b0f9d9515aaad22...,108,2022-09-30 23:46:52,5400,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,69839531a94bf2468744c083a7bf4b78f39c8b1b299c34...,desktop,no
4,ff97a5e4ea0bb411f72e2e1750fb94b248ec2026406a3f...,6efdbec56858f25686797c974bfa67809a718408bd3631...,1314,2022-09-30 23:16:49,10380,"filmingo v2.1 (83); iOS 15.6.1; iPad8,11",652d233c8526ebefb7a8e939ca546fe1a76498775b682a...,mobile,yes


# EDA part

In [38]:
import sql_functions as sf
schema = 'capstone_filmingo'

In [40]:
sf.get_dataframe(f' SELECT DISTINCT * FROM {schema}.playbacks WHERE device IS NULL')



,playback_ID,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,device,app user
